In [1]:
# Import libraries
# main splat import
import splat
import splat.plot as splot
import splat.photometry as sphot
import splat.empirical as spem
import splat.model

# other useful imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import astropy.units as u
from astropy.io import fits
from astropy.utils.data import download_file

Adding 2404 sources from /home/khryztophermena/python/splat//resources/Spectra/Public/SPEX-PRISM/ to spectral database
Adding 89 sources from /home/khryztophermena/python/splat//resources/Spectra/Public/MAGE/ to spectral database
Adding 145 sources from /home/khryztophermena/python/splat//resources/Spectra/Public/LRIS-RED/ to spectral database

No folders containing spectral models were found to be present


In [2]:
## Classify class that contains methods to simplify the process of reducing data
class Classify:
    
    def __init__(self, path, file, file_type='fits', name=''):
        '''
        :Purpose:
        
            To create an instance variable within the class Classify with the required parameters to successfully 
            utilize the splat method used the methods contained within
        
        :Required Parameters:
        
            :param path: The path to the folder that contains the file(s) being analyzed
            :param file: The name of the file being analyzed
            
        :Optional Parameters:
        
            :param file_type: The type of file that is being analyzed
            :param name: The name of the object observed within the file used to facilitate plotting and referencing
            
        :Output:
        
            N/a, constructor
            
        :Example:
        
            file = Classify('/home/Desktop/', 'spex_prism_0412+1044_160930.fits', name='0412+1044')
        
        '''
        path = path
        filename = path + file
        self.file = filename
        self.file_type = file_type
        self.name = name
        
        
    
    def classifyByStandard(self):
        '''
        :Purpose:
        
            Determine the spectral type and uncertainty for a spectrum by direct comparison to defined spectral
            standards. Dwarf standards span M0-T9 and include the standards listed in `Burgasser et al. (2006) 
            <http://adsabs.harvard.edu/abs/2006ApJ...637.1067B>`_, `Kirkpatrick et al. (2010) 
            <http://adsabs.harvard.edu/abs/2010ApJS..190..100K>`_ and `Cushing et al. (2011) 
            <http://adsabs.harvard.edu/abs/2011ApJ...743...50C>`_. 
            Comparison to subdwarf and extreme subdwarf standards may also be done.
            Returns the best match or an F-test weighted mean and uncertainty. There is an option
            to follow the procedure of `Kirkpatrick et al. (2010)
            <http://adsabs.harvard.edu/abs/2010ApJS..190..100K>`_, fitting only in
            the 0.9-1.4 micron region.

        :Required Parameters:
        
            N/a, No parameters are required but the calling object must be an instance variable of the class 
            Classify
        
        :Optional Parameters:
        
            N/a, No parameters are required but the calling object must be an instance variable of the class 
            Classify

        :Output:
        
             A tuple listing the best match standard and uncertainty based on F-test weighting and systematic uncertainty of 0.5 subtypes
             
        :Example:
        
            import splat
            file = Classify('/home/Desktop/', 'spex_prism_0412+1044_160930.fits', name='0412+1044')
            starType = file.classifyByStandard()
            
        '''
        sp1 = splat.Spectrum(file=self.file, file_type=self.file_type, name=self.name)
        result = splat.classifyByStandard(sp1)
        return result[0], result[1]
    

    
    def classifyByIndex(self):
        '''
        :Purpose:
        
            Determine the spectral type and uncertainty for a spectrum based on indices. Makes use of published 
            index-SpT relations from `Reid et al. (2001) <http://adsabs.harvard.edu/abs/2001AJ....121.1710R>`_; 
            `Testi et al. (2001) <http://adsabs.harvard.edu/abs/2001ApJ...552L.147T>`_; `Allers et al. (2007) 
            <http://adsabs.harvard.edu/abs/2007ApJ...657..511A>`_; and `Burgasser (2007) 
            <http://adsabs.harvard.edu/abs/2007ApJ...659..655B>`_. Returns 2-element tuple containing spectral 
            type (numeric or string) and uncertainty.
            
        :Required Parameters:
        
            N/a, No parameters are required but the calling object must be an instance variable of the class 
            Classify
        
        :Optional Parameters:
        
            N/a, No parameters are required but the calling object must be an instance variable of the class 
            Classify

        :Output:
        
            The index of the star
            
        :Example:
        
            import splat
            file = Classify('/home/Desktop/', 'spex_prism_0412+1044_160930.fits', name='0412+1044')
            starType = file.classifyByIndex()

        '''
        sp1 = splat.Spectrum(file=self.file, file_type=self.file_type, name=self.name)
        result = splat.classifyByIndex(sp1)
        return result[0]
    
 
        
    def classifyGravity(self):
        '''
        :Purpose:
        
            Determine the gravity classification of a brown dwarf using the method of `Allers & Liu (2013) 
            <http://adsabs.harvard.edu/abs/2013ApJ...772...79A>`_. 

        :Parameters:
        
            N/a, No parameters are required but the calling object must be an instance variable of the class 
            Classify
            
        :Output:
        
            Either a string specifying the gravity classification or a dictionary specifying the gravity scores for each index

        :Example:
        
            import splat
            file = Classify('/home/Desktop/', 'spex_prism_0412+1044_160930.fits', name='0412+1044')
            starType = file.classifyGravity()
            
        '''
        sp1 = splat.Spectrum(file=self.file, file_type=self.file_type, name=self.name)
        result = splat.classifyGravity(sp1)
        return result
    

    
    def measureIndex(self, num_init, num_fin, den_init, den_fin):
        '''
        :Purpose:

            Measure an index on a spectrum based on defined methodology
            measure method can be mean, median, integrate
            index method can be ratio = 1/2, valley = 1-2/3, OTHERS

        :Required Parameters:

            :param num_init: The initial value for the range in the numerator
            :param num_fin: The final value for the range in the numerator
            :param den_init: The initial value for the range in the denominator
            :param den_fin: The final value for the range in the denominator

        :Output:

            Index value and uncertainty

        :Example:

            import splat
            file = Classify('/home/Desktop/', 'spex_prism_0412+1044_160930.fits', name='0412+1044')
            starType = file.measureIndex(1.1, 1.2, 1.22, 1.32)
            
        '''
        
        sp1 = splat.Spectrum(file=self.file, file_type=self.file_type, name=self.name)
        result = splat.measureIndex(sp1, [[num_init, num_fin],[den_init, den_fin]])
        return result
    
    
    
    def compareSpectrum_chisqr(self):
        '''
        :Purpose:

            Compare two spectra against each other within the same category of star with the 
            statistic='chisqr'
            
        :Output:
        
            The value of the desired statistic as well as the optimal scale factor. 

        :Example:
        
            import splat
            file = Classify('/home/Desktop/', 'spex_prism_0412+1044_160930.fits', name='0412+1044')
            starType = file.compareSpectrum_chisqr()
            
        '''
        sp1 = splat.Spectrum(file=self.file, file_type=self.file_type, name=self.name)
        result = splat.classifyByStandard(sp1)
        sp2 = splat.getSpectrum(spt=result[0], lucky=True)[0]
        sp1.normalize()
        sp2.normalize()
        
        chisqr = splat.compareSpectra(sp1, sp2, statistic='chisqr')
        
        return chisqr
    
    

    def compareSpectrum_stddev(self):
        '''
        :Purpose:

            Compare two spectra against each other within the same category of star with the 
            statistic='stddev'
            
        :Output:
        
            The value of the desired statistic as well as the optimal scale factor. 

        :Example:
        
            import splat
            file = Classify('/home/Desktop/', 'spex_prism_0412+1044_160930.fits', name='0412+1044')
            starType = file.compareSpectrum_stddev()
            
        '''
        sp1 = splat.Spectrum(file=self.file, file_type=self.file_type, name=self.name)
        result = splat.classifyByStandard(sp1)
        sp2 = splat.getSpectrum(spt=result[0], lucky=True)[0]
        sp1.normalize()
        sp2.normalize()
        
        stddev = splat.compareSpectra(sp1, sp2, statistic='stddev')
        
        return stddev
    
    
    
    def compareSpectrum_absdev(self):
        '''
        :Purpose:

            Compare two spectra against each other within the same category of star with the 
            statistic='absdev'
            
        :Output:
        
            The value of the desired statistic as well as the optimal scale factor. 

        :Example:
        
            import splat
            file = Classify('/home/Desktop/', 'spex_prism_0412+1044_160930.fits', name='0412+1044')
            starType = file.compareSpectrum_absdev()
            
        '''
        sp1 = splat.Spectrum(file=self.file, file_type=self.file_type, name=self.name)
        result = splat.classifyByStandard(sp1)
        sp2 = splat.getSpectrum(spt=result[0], lucky=True)[0]
        sp1.normalize()
        sp2.normalize()
        
        absdev = splat.compareSpectra(sp1, sp2, statistic='absdev')
        
        return absdev